Created a Kaggle directory (~/.kaggle) and copied my Kaggle API key file (kaggle.json) into this directory.

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 /root/.kaggle/kaggle.json

Since the dataset is a huge file we can use the Kaggle API key to download the complete dataset here directly.

In [ ]:
!kaggle datasets download -d salader/dogs-vs-cats

100% 1.06G/1.06G [00:37<00:00, 32.8MB/s]
100% 1.06G/1.06G [00:37<00:00, 30.8MB/s]


extracting contents of the zipped file

In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/content/dogs-vs-cats.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,BatchNormalization,Dropout


*   Generators in Keras are Python iterators that yield batches of data during training instead of loading the entire dataset into memory at once.

*   the image_datsets_from_directory in keras is used to create the training and validation datasets from image directories.Further all images are resized into a uniform pixel size of 256x256

In [ ]:
train_ds = keras.utils.image_dataset_from_directory(
    directory = '/content/train',
    labels='inferred',
    label_mode = 'int',
    batch_size=32,
    image_size=(256,256)
)

validation_ds = keras.utils.image_dataset_from_directory(
    directory = '/content/test',
    labels='inferred',
    label_mode = 'int',
    batch_size=32,
    image_size=(256,256)
)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.


We carry out normalization since the images in the dataset are in the pixel range of 0-255 we need to bring it down to the range of 0-1 for better evaluation.

In [ ]:
def process(image,label):
    image = tf.cast(image/255. ,tf.float32)
    return image,label

train_ds = train_ds.map(process)
validation_ds = validation_ds.map(process)

# 3 layer CNN model
Summary of the architecture:
*   Three sets of Convolutional Layers, each followed by Batch Normalization and MaxPooling2D.
*   Flatten layer to transition from convolutional layers to dense layers.
*   Two Dense (Fully Connected) Layers with ReLU activation and dropout for regularization.
*   Output layer with a single neuron and sigmoid activation for binary classification.

In [ ]:
model = Sequential()

# First Convolutional Layer
model.add(Conv2D(32,kernel_size=(3,3),padding='valid',activation='relu',input_shape=(256,256,3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

# Second Convolutional Layer
model.add(Conv2D(64,kernel_size=(3,3),padding='valid',activation='relu',input_shape=(256,256,3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

# Third Convolutional Layer
model.add(Conv2D(128,kernel_size=(3,3),padding='valid',activation='relu',input_shape=(256,256,3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))


# Flatten layer to transition from convolutional layers to dense layers
model.add(Flatten())

# Dense (Fully Connected) Layers
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.1))

model.add(Dense(64,activation='relu'))
model.add(Dropout(0.1))


# Output layer with sigmoid activation for binary classification
model.add(Dense(1,activation='sigmoid'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 32)      896       
                                                                 
 batch_normalization (Batch  (None, 254, 254, 32)      128       
 Normalization)                                                  
                                                                 
 max_pooling2d (MaxPooling2  (None, 127, 127, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 64)      18496     
                                                                 
 batch_normalization_1 (Bat  (None, 125, 125, 64)      256       
 chNormalization)                                                
                                                        

# Model Compilation
1. Optimizer ('adam'): The Adam optimizer adapts to the learning rates during training.

2. Loss ('binary_crossentropy'): Binary crossentropy is a suitable loss function for binary classification problems. It measures the difference between the true labels and the predicted probabilities.

3. Metrics (['accuracy']): the accuracy metric to evaluate how well your model is performing on the training data

In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
result = model.fit(train_ds,epochs=10,validation_data=validation_ds)

Epoch 1/10
625/625 [==============================] - 4062s 6s/step - loss: 1.3583 - accuracy: 0.5624 - val_loss: 0.6756 - val_accuracy: 0.5974
Epoch 2/10
215/625 [=========>....................] - ETA: 41:09 - loss: 0.6710 - accuracy: 0.6062

In [ ]:
import matplotlib.pyplot as plt

# Plot training & validation accuracy values
plt.plot(result.history['accuracy'])
plt.plot(result.history['val_accuracy'])
plt.title('Model accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()


In [ ]:
# Plot training & validation loss values
plt.plot(result.history['loss'])
plt.plot(result.history['val_loss'])
plt.title('Model loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()

(Open-source computer vision) OpenCV and image processing library is used for image processing tasks such as reading and displaying images, resizing, applying filters, and more.

In [ ]:
import cv2

In [ ]:
test_img = cv2.imread('/content/dog.jpg')

In [ ]:
plt.imshow(test_img)

In [ ]:
test_img.shape

In [ ]:
test_img = cv2.resize(test_img,(256,256))
test_input = test_img.reshape((1,256,256,3))

In [ ]:
model.predict(test_input)